In [1]:
import sys
import os
import logging
from kafka import KafkaConsumer
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import col, explode, from_json, udf, when,  abs as spark_abs, array
from pyspark.sql.types import StructType, StructField, StringType
from pyspark.sql.types import ArrayType, MapType, IntegerType, FloatType, BooleanType, DoubleType
import re
import json

In [52]:
spark = SparkSession.builder \
    .appName("KafkaConsumer") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.4.0") \
    .getOrCreate()



In [53]:
# parsed_df = spark.read.csv('data/ouput/raw_data.csv', header=True)
# df = spark.read.option("quote", "\"").option("multiline", "true").csv('data/ouput/raw_data.csv')
parsed_df = spark.read.option("quote", "\"") \
               .option("multiline", "true") \
               .option("escape", "\"") \
               .csv('data/ouput/raw_data.csv', header=True)
parsed_df.show(5, truncate=False)

+----------------------------------------------------------------------+---------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---

In [54]:
renamed_columns = {
        "Tên sản phẩm": "ten",
        "Loại điện thoại": "loai_dien_thoai",
        "Màu sắc - Phiên bản bộ nhớ - Giá tương ứng": "mau_sac_phien_ban_bo_nho_gia_moi_gia_cu",
        "Thời gian bảo hành": "thoi_gian_bao_hanh",
        "Thông số kỹ thuật": "thong_so_ky_thuat",
        "Đánh giá": "danh_gia",
        "Số lượt đánh giá và hỏi đáp": "so_luong_binh_luan",
        "Đường dẫn": "duong_dan"
    }

for old_name, new_name in renamed_columns.items():
    parsed_df = parsed_df.withColumnRenamed(old_name, new_name)
    
    # Loại bỏ các hàng có giá trị "ten" không mong muốn
parsed_df = parsed_df.filter(col("ten") != "Điện thoại Xiaomi SU7 (Ô tô điện chạy 800km/1 lần sạc)")

    # Loại bỏ các hàng có giá trị "loai_dien_thoai" không mong muốn
parsed_df = parsed_df.filter((col("loai_dien_thoai") != "Apple Watch") & 
                                (col("loai_dien_thoai") != "Máy Chơi Game PC"))
    

In [55]:
parsed_df.show(5, truncate=False)

+----------------------------------------------------------------------+---------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---

In [57]:
spark.stop()

In [56]:
parsed_df = parsed_df.withColumn(
    "mau_sac_phien_ban_bo_nho_gia_moi_gia_cu",
    F.from_json(
        F.to_json(F.col("mau_sac_phien_ban_bo_nho_gia_moi_gia_cu")),  # <<< CHỈ THÊM to_json()
        ArrayType(ArrayType(StringType()))
    )
)

parsed_df = parsed_df.withColumn(
    "exploded",
    F.explode("mau_sac_phien_ban_bo_nho_gia_moi_gia_cu")
)

parsed_df = parsed_df.withColumn("mau_sac", F.col("exploded")[0]) \
                     .withColumn("phien_ban_bo_nho", F.col("exploded")[1]) \
                     .withColumn("gia_moi", F.col("exploded")[2]) \
                     .withColumn("gia_cu", F.col("exploded")[3]) \
                     .drop("exploded", "mau_sac_phien_ban_bo_nho_gia_moi_gia_cu")

AnalysisException: [DATATYPE_MISMATCH.INVALID_JSON_SCHEMA] Cannot resolve "to_json(mau_sac_phien_ban_bo_nho_gia_moi_gia_cu)" due to data type mismatch: Input schema "STRING" must be a struct, an array or a map.;
'Project [ten#3749, loai_dien_thoai#3758, from_json(ArrayType(ArrayType(StringType,true),true), to_json(mau_sac_phien_ban_bo_nho_gia_moi_gia_cu#3767, Some(Asia/Bangkok)), Some(Asia/Bangkok)) AS mau_sac_phien_ban_bo_nho_gia_moi_gia_cu#3862, thoi_gian_bao_hanh#3776, thong_so_ky_thuat#3785, danh_gia#3794, so_luong_binh_luan#3803, duong_dan#3812]
+- Filter (NOT (loai_dien_thoai#3758 = Apple Watch) AND NOT (loai_dien_thoai#3758 = Máy Chơi Game PC))
   +- Filter NOT (ten#3749 = Điện thoại Xiaomi SU7 (Ô tô điện chạy 800km/1 lần sạc))
      +- Project [ten#3749, loai_dien_thoai#3758, mau_sac_phien_ban_bo_nho_gia_moi_gia_cu#3767, thoi_gian_bao_hanh#3776, thong_so_ky_thuat#3785, danh_gia#3794, so_luong_binh_luan#3803, Đường dẫn#3698 AS duong_dan#3812]
         +- Project [ten#3749, loai_

In [32]:
parsed_df.select('duong_dan').show(5, truncate=False)

+----------------------------------------------------------------+
|duong_dan                                                       |
+----------------------------------------------------------------+
|https://mobilecity.vn/dien-thoai/xiaomi-redmi-12c-helio-g85.html|
|https://mobilecity.vn/dien-thoai/xiaomi-redmi-12c-helio-g85.html|
|https://mobilecity.vn/dien-thoai/xiaomi-redmi-12c-helio-g85.html|
|https://mobilecity.vn/dien-thoai/xiaomi-redmi-12c-helio-g85.html|
|https://mobilecity.vn/dien-thoai/xiaomi-redmi-12c-helio-g85.html|
+----------------------------------------------------------------+
only showing top 5 rows



In [21]:
parsed_df.count()

10069

In [33]:
parsed_df = parsed_df.withColumn(
    "thong_so_ky_thuat",
    from_json(col("thong_so_ky_thuat"), MapType(StringType(), StringType()))
)
# 2. Tạo các cột mới từ các phần tử trong array
parsed_df = parsed_df.withColumn("man_hinh",        col("thong_so_ky_thuat")["Màn hình:"]) \
                     .withColumn("he_dieu_hanh",     col("thong_so_ky_thuat")["Hệ điều hành:"]) \
                     .withColumn("camera_sau",       col("thong_so_ky_thuat")["Camera sau:"]) \
                     .withColumn("camera_truoc",     col("thong_so_ky_thuat")["Camera trước:"]) \
                     .withColumn("cpu",              col("thong_so_ky_thuat")["CPU:"]) \
                     .withColumn("ram",              col("thong_so_ky_thuat")["RAM:"]) \
                     .withColumn("bo_nho_trong",     col("thong_so_ky_thuat")["Bộ nhớ trong:"]) \
                     .withColumn("the_sim",          col("thong_so_ky_thuat")["Thẻ SIM:"]) \
                     .withColumn("dung_luong_pin",   col("thong_so_ky_thuat")["Dung lượng pin:"]) \
                     .withColumn("thiet_ke",         col("thong_so_ky_thuat")["Thiết kế:"])
# 3. Xóa cột 'thong_so_ky_thuat' gốc
parsed_df = parsed_df.drop("thong_so_ky_thuat")

parsed_df.show(5)

+--------------------+---------------+------------------+--------+------------------+--------------------+-----------+----------------+-----------+----------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------+------------------+---------------+--------------------+---------------+
|                 ten|loai_dien_thoai|thoi_gian_bao_hanh|danh_gia|so_luong_binh_luan|           duong_dan|    mau_sac|phien_ban_bo_nho|    gia_moi|    gia_cu|            man_hinh|       he_dieu_hanh|          camera_sau|        camera_truoc|                 cpu|           ram|      bo_nho_trong|        the_sim|      dung_luong_pin|       thiet_ke|
+--------------------+---------------+------------------+--------+------------------+--------------------+-----------+----------------+-----------+----------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------+------------------+----

In [34]:
parsed_df.show(5, truncate=False)

+---------------------------------------+---------------+------------------+--------+------------------+----------------------------------------------------------------+-----------+----------------+-----------+----------+-------------------------------------------------------------+-------------------+----------------------------------------------------------------+----------------------------+------------------------------------------------------------------------------------+--------------+------------------+---------------+-----------------------+---------------+
|ten                                    |loai_dien_thoai|thoi_gian_bao_hanh|danh_gia|so_luong_binh_luan|duong_dan                                                       |mau_sac    |phien_ban_bo_nho|gia_moi    |gia_cu    |man_hinh                                                     |he_dieu_hanh       |camera_sau                                                      |camera_truoc                |cpu                           

In [36]:
# Định nghĩa hàm get_brand
def get_brand(s):
    if not isinstance(s, str):
        return None
    s = s.replace("Điện thoại", "")
    s = s.strip()
    res = s.split(" ")[0].strip()
    if res == "OPPO":
        return "Oppo"
    if "Bkav" in res:
        return "Bphone"
    if res == "Thành":
        return "Thành Hưng"
    return res
get_brand_udf = udf(get_brand, StringType())
parsed_df = parsed_df.withColumn("hang_dien_thoai", get_brand_udf(col("ten")))
parsed_df.show(5)


+--------------------+---------------+------------------+--------+------------------+--------------------+-----------+----------------+-----------+----------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------+------------------+---------------+--------------------+---------------+---------------+
|                 ten|loai_dien_thoai|thoi_gian_bao_hanh|danh_gia|so_luong_binh_luan|           duong_dan|    mau_sac|phien_ban_bo_nho|    gia_moi|    gia_cu|            man_hinh|       he_dieu_hanh|          camera_sau|        camera_truoc|                 cpu|           ram|      bo_nho_trong|        the_sim|      dung_luong_pin|       thiet_ke|hang_dien_thoai|
+--------------------+---------------+------------------+--------+------------------+--------------------+-----------+----------------+-----------+----------+--------------------+-------------------+--------------------+--------------------+--------------------+------

In [38]:
# Định nghĩa hàm isOld
def isOld(s):
    if not isinstance(s, str):
        return None
    s = s.lower()
    return "cũ" in s

isOld_udf = udf(isOld, BooleanType())

parsed_df = parsed_df.withColumn("old1", isOld_udf(col("ten")))
parsed_df = parsed_df.withColumn("old2", isOld_udf(col("loai_dien_thoai")))

parsed_df = parsed_df.withColumn("la_dien_thoai_cu", col("old1") | col("old2"))

parsed_df = parsed_df.drop("old1", "old2")
parsed_df.show(5, truncate=False)

+---------------------------------------+---------------+------------------+--------+------------------+----------------------------------------------------------------+-----------+----------------+-----------+----------+-------------------------------------------------------------+-------------------+----------------------------------------------------------------+----------------------------+------------------------------------------------------------------------------------+--------------+------------------+---------------+-----------------------+---------------+---------------+----------------+
|ten                                    |loai_dien_thoai|thoi_gian_bao_hanh|danh_gia|so_luong_binh_luan|duong_dan                                                       |mau_sac    |phien_ban_bo_nho|gia_moi    |gia_cu    |man_hinh                                                     |he_dieu_hanh       |camera_sau                                                      |camera_truoc              

In [39]:
def cast_price(s):
    if not isinstance(s, str):
        return None  # Tương đương với np.nan trong Pandas
    s = s.replace(".", "")
    numbers = re.findall(r'\d+', s)
    if len(numbers) == 0:
        return None  # Tương đương với np.nan trong Pandas
    return int(numbers[0])
cast_price_udf = udf(cast_price, IntegerType())

parsed_df = parsed_df.withColumn("gia_moi", cast_price_udf(col("gia_moi")))
parsed_df = parsed_df.withColumn("gia_cu", cast_price_udf(col("gia_cu")))
parsed_df.show(5, truncate=False)


+---------------------------------------+---------------+------------------+--------+------------------+----------------------------------------------------------------+-----------+----------------+-------+-------+-------------------------------------------------------------+-------------------+----------------------------------------------------------------+----------------------------+------------------------------------------------------------------------------------+--------------+------------------+---------------+-----------------------+---------------+---------------+----------------+
|ten                                    |loai_dien_thoai|thoi_gian_bao_hanh|danh_gia|so_luong_binh_luan|duong_dan                                                       |mau_sac    |phien_ban_bo_nho|gia_moi|gia_cu |man_hinh                                                     |he_dieu_hanh       |camera_sau                                                      |camera_truoc                |cpu        

In [40]:
# Định nghĩa hàm cast_warranty
def cast_warranty(s):
    if not isinstance(s, str):
        return None  # Tương đương với np.nan trong Pandas
    numbers = re.findall(r'\d+', s)
    if len(numbers) == 0:
        return None  # Tương đương với np.nan trong Pandas
    if len(numbers) == 1:
        if numbers[0] == "30":
            return 1
        return int(numbers[0])
    if numbers[0] == "6" and numbers[1] == "12":
        return 12
    if numbers[0] == "12" and numbers[1] == "12":
        return 24
    return int(numbers[0])
cast_warranty_udf = udf(cast_warranty, IntegerType())
parsed_df = parsed_df.withColumn("thoi_gian_bao_hanh", cast_warranty_udf(col("thoi_gian_bao_hanh")))
parsed_df.show(5, truncate=False)

+---------------------------------------+---------------+------------------+--------+------------------+----------------------------------------------------------------+-----------+----------------+-------+-------+-------------------------------------------------------------+-------------------+----------------------------------------------------------------+----------------------------+------------------------------------------------------------------------------------+--------------+------------------+---------------+-----------------------+---------------+---------------+----------------+
|ten                                    |loai_dien_thoai|thoi_gian_bao_hanh|danh_gia|so_luong_binh_luan|duong_dan                                                       |mau_sac    |phien_ban_bo_nho|gia_moi|gia_cu |man_hinh                                                     |he_dieu_hanh       |camera_sau                                                      |camera_truoc                |cpu        

In [41]:
def cast_rating(s):
    if not isinstance(s, str):
        return None  # Tương đương với np.nan trong Pandas
    numbers = re.findall(r"-?\d+(?:\.\d+)?", s)
    if len(numbers) == 0:
        return None  # Tương đương với np.nan trong Pandas
    return float(numbers[0])
cast_rating_udf = udf(cast_rating, FloatType())
parsed_df = parsed_df.withColumn("danh_gia", cast_rating_udf(col("danh_gia")))
parsed_df.show(5, truncate=False)

+---------------------------------------+---------------+------------------+--------+------------------+----------------------------------------------------------------+-----------+----------------+-------+-------+-------------------------------------------------------------+-------------------+----------------------------------------------------------------+----------------------------+------------------------------------------------------------------------------------+--------------+------------------+---------------+-----------------------+---------------+---------------+----------------+
|ten                                    |loai_dien_thoai|thoi_gian_bao_hanh|danh_gia|so_luong_binh_luan|duong_dan                                                       |mau_sac    |phien_ban_bo_nho|gia_moi|gia_cu |man_hinh                                                     |he_dieu_hanh       |camera_sau                                                      |camera_truoc                |cpu        

In [42]:
def cast_memory_version(s):
    if not isinstance(s, str):
        return [None, None]  # Tương đương với [np.nan, np.nan]
    numbers = re.findall(r"-?\d+(?:\.\d+)?", s)
    if len(numbers) == 0:
        return [None, None]
    if len(numbers) == 1:
        if abs(float(numbers[0])) < 4:
            return [None, abs(float(numbers[0]) * 1024)]
        return [None, abs(float(numbers[0]))]
    if abs(float(numbers[1])) < 4:
        return [abs(float(numbers[0])), abs(float(numbers[1]) * 1024)]
    return [abs(float(numbers[0])), abs(float(numbers[1]))]

cast_memory_version_udf = udf(cast_memory_version, ArrayType(DoubleType()))

parsed_df = parsed_df.withColumn("phien_ban_bo_nho", cast_memory_version_udf(col("phien_ban_bo_nho")))

parsed_df = parsed_df.withColumn("_ram", col("phien_ban_bo_nho")[0])
parsed_df = parsed_df.withColumn("_bo_nho_trong", col("phien_ban_bo_nho")[1])
parsed_df = parsed_df.drop("phien_ban_bo_nho")
parsed_df.show(5, truncate=False)

+---------------------------------------+---------------+------------------+--------+------------------+----------------------------------------------------------------+-----------+-------+-------+-------------------------------------------------------------+-------------------+----------------------------------------------------------------+----------------------------+------------------------------------------------------------------------------------+--------------+------------------+---------------+-----------------------+---------------+---------------+----------------+----+-------------+
|ten                                    |loai_dien_thoai|thoi_gian_bao_hanh|danh_gia|so_luong_binh_luan|duong_dan                                                       |mau_sac    |gia_moi|gia_cu |man_hinh                                                     |he_dieu_hanh       |camera_sau                                                      |camera_truoc                |cpu                       

In [43]:

# Hàm xử lý RAM
def firstNumberRAM(s):
    if not isinstance(s, str):
        return None  # Tương đương với np.nan trong Pandas
    numbers = re.findall(r"-?\d+(?:\.\d+)?", s)
    if len(numbers) == 0:
        return None
    res = float(numbers[0])
    return res if res < 100 else res * 1.0 / 1024

# Hàm xử lý ROM
def firstNumberROM(s):
    if not isinstance(s, str):
        return None  # Tương đương với np.nan trong Pandas
    numbers = re.findall(r"-?\d+(?:\.\d+)?", s)
    if len(numbers) == 0:
        return None
    res = float(numbers[0])
    if res < 2:
        res = res * 1024
    return res

firstNumberRAM_udf = udf(firstNumberRAM, DoubleType())
firstNumberROM_udf = udf(firstNumberROM, DoubleType())

parsed_df = parsed_df.withColumn("ram", firstNumberRAM_udf(col("ram")))
parsed_df = parsed_df.withColumn("bo_nho_trong", firstNumberROM_udf(col("bo_nho_trong")))

parsed_df = parsed_df.withColumn("ram", when(col("ram").isNull(), col("_ram")).otherwise(col("ram")))
parsed_df = parsed_df.withColumn("bo_nho_trong", when(col("bo_nho_trong").isNull(), col("_bo_nho_trong")).otherwise(col("bo_nho_trong")))

parsed_df = parsed_df.drop("_ram", "_bo_nho_trong")


In [44]:
parsed_df.show(5, truncate=False)

+---------------------------------------+---------------+------------------+--------+------------------+----------------------------------------------------------------+-----------+-------+-------+-------------------------------------------------------------+-------------------+----------------------------------------------------------------+----------------------------+------------------------------------------------------------------------------------+---+------------+---------------+-----------------------+---------------+---------------+----------------+
|ten                                    |loai_dien_thoai|thoi_gian_bao_hanh|danh_gia|so_luong_binh_luan|duong_dan                                                       |mau_sac    |gia_moi|gia_cu |man_hinh                                                     |he_dieu_hanh       |camera_sau                                                      |camera_truoc                |cpu                                                           

In [ ]:

# Hàm xử lý RAM
def firstNumberRAM(s):
    if not isinstance(s, str):
        return None  # Tương đương với np.nan trong Pandas
    numbers = re.findall(r"-?\d+(?:\.\d+)?", s)
    if len(numbers) == 0:
        return None
    res = float(numbers[0])
    return res if res < 100 else res * 1.0 / 1024

# Hàm xử lý ROM
def firstNumberROM(s):
    if not isinstance(s, str):
        return None  # Tương đương với np.nan trong Pandas
    numbers = re.findall(r"-?\d+(?:\.\d+)?", s)
    if len(numbers) == 0:
        return None
    res = float(numbers[0])
    if res < 2:
        res = res * 1024
    return res


# Hàm trích xuất kích thước màn hình (inches)
def extractInches(s):
    if not isinstance(s, str):
        return None
    s = s.lower()
    match = re.search(r"(\d+[.,]?\d*)\s*inches", s)
    result = float(match.group(1).replace(",", ".")) if match else None
    return result

# Hàm trích xuất tần số quét (Hz)
def extractHz(s):
    if not isinstance(s, str):
        return None
    s = s.lower()
    match = re.search(r"(\d+(\.\d+)?)\s*hz", s)
    result = float(match.group(1)) if match else None
    return result

# Hàm trích xuất độ sáng màn hình (nits)
def extractNits(s):
    if not isinstance(s, str):
        return None
    s = s.lower()
    match = re.search(r"(\d+(\.\d+)?)\s*nits", s)
    result = float(match.group(1)) if match else None
    if result and result < 50:
        result = result * 1000
    return result

# Hàm trích xuất loại màn hình
def extractType(s):
    if not isinstance(s, str):
        return None
    s = s.upper()
    if "AMOLED" in s:
        return "AMOLED"
    if "OLED" in s:
        return "OLED"
    if "LCD" in s:
        return "LCD"
    return None


# Định nghĩa hàm extractSim
def extractSim(s):
    if not isinstance(s, str):
        return None  # Tương đương với np.nan trong Pandas
    s = s.lower()
    if "2" in s or "dual" in s or "kép" in s:
        return 2
    if "không" in s:
        return 0
    return 1


# Hàm trích xuất loại pin
def batteryType(s):
    if not isinstance(s, str):
        return ""
    s = s.lower()
    if "li-po" in s:
        return "Li-Po"
    if "li-ion" in s:
        return "Li-Ion"
    if "si/c" in s:
        return "Si/C"
    return ""

# Hàm trích xuất dung lượng pin
def batteryCapacity(s):
    if not isinstance(s, str):
        return None  # Tương đương với np.nan trong Pandas
    s = s.lower()
    match = re.search(r"(\d+(\.\d+)?)\s*mah", s)
    result = float(match.group(1)) if match else None
    return result

# Hàm trích xuất công suất sạc
def chargingCapacity(s):
    if not isinstance(s, str):
        return None  # Tương đương với np.nan trong Pandas
    s = s.lower()
    match = re.search(r"(\d+(\.\d+)?)\s*w", s)
    result = float(match.group(1)) if match else None
    return result



# Hàm trích xuất độ phân giải camera
def extractMP(s):
    if not isinstance(s, str):
        return []  # Tương đương với danh sách trống
    s = s.upper()
    matches = re.findall(r"(\d+(\.\d+)?)\s*MP", s)
    result = [float(match[0]) for match in matches]
    return result


# Hàm chuyển đổi hệ điều hành
def cast_OS(s):
    if not isinstance(s, str):
        return "Khác"
    s = s.lower()
    numbers = re.findall(r"\d+", s)
    if len(numbers) == 0:
        return "Khác"
    if "android" in s:
        return "Android " + str(int(numbers[0]))
    if "ios" in s:
        return "iOS " + str(int(numbers[0]))
    return "Khác"

In [5]:
from kafka import KafkaConsumer
from kafka import KafkaProducer, TopicPartition
# Đọc dữ liệu từ topic "mobile_topic"
consumer = KafkaConsumer(
    'mobile_topic',
    bootstrap_servers='localhost:9092',  # Địa chỉ Kafka broker
    auto_offset_reset='earliest',  # Đọc từ đầu
    enable_auto_commit=True,
    group_id='test_consumer_group'
)

print("Reading messages from Kafka topic 'mobile_topic'...")
for message in consumer:
    print(f"Key: {message.key}, Value: {message.value.decode('utf-8')}")

NoBrokersAvailable: NoBrokersAvailable

In [8]:
from kafka import KafkaConsumer

# Cấu hình Kafka
bootstrap_servers = 'kafka:9092'  # Địa chỉ Kafka
topic = 'mobile_topic'  # Tên topic cần kiểm tra

# Tạo Kafka Consumer
consumer = KafkaConsumer(
    topic,
    bootstrap_servers=bootstrap_servers,
    auto_offset_reset='earliest',  # Đọc từ đầu
    enable_auto_commit=True,
    group_id='test_consumer_group'
)

# Đọc và hiển thị dữ liệu
print(f"Reading messages from Kafka topic '{topic}'...")
for message in consumer:
    print(f"Key: {message.key}, Value: {message.value.decode('utf-8')}")

NoBrokersAvailable: NoBrokersAvailable